In [18]:
import gust
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy

from scipy.spatial.distance import squareform

In [2]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

A, X, _, z = gust.load_dataset('cora_ml').standardize().unpack()
#adj = torch.FloatTensor(A.toarray()).cuda()

In [3]:
N = A.shape[0]
D = 32

Z = nn.Parameter(torch.empty(N, D).normal_(std=0.1))
x = nn.Parameter(torch.empty(N, D).normal_(std=0.1))

opt = torch.optim.Adam([Z], lr=1e-2)
e1, e2 = A.nonzero()

In [4]:
def L0(A):    
    return torch.FloatTensor(A.toarray()).cuda()

In [5]:
def L1(A):
    #L=D^-1A
    degreenp=A.sum(axis=1)
    degree = torch.from_numpy(degreenp).cuda().view(-1)
    degreematrix = torch.zeros(N,N)
    degreematrix[np.diag_indices(N)]=degree
    invdegree = torch.inverse(degreematrix)
    adj = torch.FloatTensor(A.toarray()).cuda()
    return torch.matmul(invdegree, adj)

Z = L1(A)

In [23]:
def L2(A):
    degreenp=A.sum(axis=1)
    degree = torch.from_numpy(degreenp).cuda().view(-1)
    degreematrix = torch.zeros(N,N)
    degreematrix[np.diag_indices(N)]=degree
    sqrtdegree = degreematrix.sqrt()
    invdegree = torch.inverse(sqrtdegree)
    adj = torch.FloatTensor(A.toarray()).cuda() 
    return torch.matmul(invdegree, torch.matmul(adj, invdegree))

In [24]:
L2(A)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1429,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1429, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [20]:
def sig(Z, b=0.1, eps=1e-8): 
    dist = torch.matmul(Z,Z.T) +b
    sigdist = 1/(1+torch.exp(dist+eps)+eps)
    logsigdist = torch.log(sigdist+eps)
    pos_term = logsigdist[e1,e2]
    neg_term = torch.log(1-sigdist)
    neg_term[np.diag_indices(N)] = 0.0
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [21]:
def dist(Z, eps=1e-5):
    gamma = 0.1
    dist = ((Z[:, None] - Z[None, :]).pow(2.0).sum(-1) + eps).sqrt()
    neg_term = torch.log(-torch.expm1(-dist)*gamma + eps)
    print(neg_term)
    neg_term[np.diag_indices(N)] = 0.0
    pos_term = -dist[e1, e2]   
    neg_term[e1, e2] = 0.0
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [22]:
def exp(Z, eps=1e-8):
    #e1, e2 = similarity_measure.nonzero()
    emb_abs = torch.FloatTensor.abs(Z)
    dist = -torch.matmul(emb_abs, emb_abs.T)
    neg_term = dist
    neg_term[np.diag_indices(Z.shape[0])] = 0.0
    expdist = torch.exp(dist)
    embedding = 1 - expdist
    logdist = torch.log(embedding + eps)
    pos_term = logdist[e1, e2]
    size=Z.shape[0]
    print(embedding)
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [23]:
def kl(L, Z, eps=1e-8):
    #P=softmax(ZZ^T)
    dist=torch.matmul(Z,Z.T)
    sigdist = 1/(1+torch.exp(dist+eps)+eps)
    logsigdist = torch.log(sigdist+eps)
    losses = T*logsigdist
    return losses.sum()

In [24]:
for epoch in range(3):
    opt.zero_grad()
    loss = exp(Z)
    loss.backward()
    opt.step()
    print(loss.item())

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0202,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0202, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn